Referências:  
http://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html  
https://numba.pydata.org/  
https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en  
https://gist.github.com/jfpuget/b53f1e15a37aba5944ad  

In [1]:
import random
import time

Criando uma função Fibonacci Recursiva:

In [2]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [5]:
%timeit fib(20)
%timeit fib(30)
%timeit fib(33)

4.64 ms ± 340 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
714 ms ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.42 s ± 251 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Compilando a função com o compilador Cython:  
(pip3 install Cython)

In [6]:
%load_ext Cython

In [7]:
%%cython

def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [8]:
%timeit fib_cython(20)
%timeit fib_cython(30)
%timeit fib_cython(33)

1.77 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
217 ms ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
925 ms ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Podemos melhorar ainda mais compilando as variáveis estaticamente. A função deve ser declarada com "cpdef" ao invés de def". Dessa forma, podemos usar os tipos do C como parâmetros da função:

In [9]:
%%cython

cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [10]:
%timeit fib_cython_type(20)
%timeit fib_cython_type(30)
%timeit fib_cython_type(33)

61.3 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.74 ms ± 703 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
27 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Uma outra abordagem é permitindo que os resultados sejam guardados em _cache_:
Dessa forma podemos manter a precisão arbitrária das variáveis do Python, sem recorrer aos [tipos estáticos do C](https://notes-on-cython.readthedocs.io/en/latest/function_declarations.html)

In [11]:
from functools import lru_cache as cache #Python3

@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

In [13]:
%timeit fib_cache(20)
%timeit fib_cache(30)
%timeit fib_cache(33)

201 ns ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
278 ns ± 2.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
200 ns ± 51.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Uma outra maneira de emular o cache seria realizando uma modificação no algoritmo:

In [14]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [15]:
%timeit fib_seq(20)
%timeit fib_seq(30)
%timeit fib_seq(33)

2.32 µs ± 568 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.39 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.6 µs ± 37.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Vamos compilar esta nova função com e sem a utilização de tipagem estática: 

In [16]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [17]:
%timeit fib_seq_cython(20)
%timeit fib_seq_cython(30)
%timeit fib_seq_cython(33)

870 ns ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.36 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.41 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%%cython

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [19]:
%timeit fib_seq_cython_type(20)
%timeit fib_seq_cython_type(30)
%timeit fib_seq_cython_type(33)

126 ns ± 30 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
122 ns ± 18 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
124 ns ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Podemos usar uma ferramenta chamada Numba, que é um just-in-time (jit) compiler:

In [20]:
from numba import jit

In [27]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [28]:
%timeit fib_seq_numba(20)
%timeit fib_seq_numba(30)
%timeit fib_seq_numba(33)

328 ns ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
330 ns ± 9.27 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
456 ns ± 50.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Segundo exemplo:  
Algoritmo de ordenação Quick Sort  

In [29]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort():
    lst = [random.random() for i in range(1,5000)]
    qsort_kernel(lst, 0, len(lst)-1)

Usando Numba:

In [30]:
@jit
def qsort_kernel_numba(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba():
        lst = [random.random() for i in range(1,5000)]
        qsort_kernel_numba(lst, 0, len(lst)-1)

Numba e Numpy:

In [31]:
@jit
def qsort_kernel_numba_numpy(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba_numpy(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba_numpy():
        lst = np.random.rand(5000)
        qsort_kernel_numba(lst, 0, len(lst)-1)

Usando Numpy e Cython:  

In [32]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] \
qsort_kernel_cython_numpy_type(double[:] a, \
                               long lo, \
                               long hi):
    cdef: 
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

In [33]:
%timeit benchmark_qsort()
%timeit benchmark_qsort_numba()
%timeit benchmark_qsort_numba_numpy()
%timeit benchmark_qsort_numpy_cython()
%timeit benchmark_sort_numpy()

20.8 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.67 ms ± 212 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
614 µs ± 664 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
820 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
536 µs ± 62.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
